In [ ]:
#0) create env
#1) pip install -r requirment.txt
#2) define your OPENAI key in  .env
#3) change datasources

# Libraries

In [34]:
import os 
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, PromptTemplate
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

# LLM

In [23]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [35]:
# LLM 
Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

In [37]:
#data
pg = SimpleDirectoryReader(
    input_files=["data/pg.pdf"]
).load_data()
ug = SimpleDirectoryReader(
    input_files=["data/ug.pdf"]
).load_data()

Generating embeddings:  89%|████████▉ | 8/9 [2:28:22<18:32, 1112.87s/it]


In [38]:
#indices
pg_index = VectorStoreIndex.from_documents(pg)
ug_index = VectorStoreIndex.from_documents(ug)


**********
Trace: index_construction
    |_embedding ->  0.791014 seconds
**********
**********
Trace: index_construction
    |_embedding ->  1.318526 seconds
**********


In [40]:
#query engine 

pg_engine = pg_index.as_query_engine(similarity_top_k=3)
ug_engine = ug_index.as_query_engine(similarity_top_k=3)


In [41]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=pg_engine,
        metadata=ToolMetadata(
            name="pg_manual",
            description=(
                "Provides information about pg manual guide for year 2023/24"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=ug_engine,
        metadata=ToolMetadata(
            name="ug_manual",
            description=(
                "Provides information about ug manual guide for year 2023/24"
            ),
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

# Run Queries

In [45]:
response = s_engine.query(
    "Compare key dates for undergraduate and postgraduate students"
)

Generated 2 sub questions.
[pg_manual] Q: What are the key dates for postgraduate students in 2023/24 according to the pg manual?
[ug_manual] Q: What are the key dates for undergraduate students in 2023/24 according to the ug manual?
[ug_manual] A: The key dates for undergraduate students in 2023/24 are as follows:
- Autumn Term: Monday 25 September – Friday 8 December 2023
- Winter break: Monday 11 December 2023 – Friday 12 January 2024
- Winter Term: Monday 15 January – Thursday 28 March 2024
- Spring break: Friday 29 March – Friday 26 April 2024
- Spring Term: Monday 29 April – Friday 14 June 2024
[pg_manual] A: Postgraduate students will be able to browse courses from 10am on Monday 18 September 2023. Course selection for postgraduates will be open from 10am on Thursday 21 September until 5pm on Friday 6 October 2023. Additionally, course selection will reopen from 10am on Monday 15 January until 5pm on Friday 26 January 2024 to allow changes to be made to Winter Term half unit cou

# The End